# Make efficiency map plots for trained fitters

In [1]:
import train as tn
reload(tn)

import plotting
reload(plotting)

import util as ut

import matplotlib.pyplot as plt
#plt.style.use('seaborn-ticks')
plt.style.use('seaborn-poster')
%matplotlib inline
#%matplotlib notebook

import numpy as np

from pprint import pprint

import itertools

# import xgboost

Welcome to ROOTaaS 6.06/08


In [2]:
#note that the loading option is True by default
ut.defaultParameters(dataDir='./data', inputName="effGenVarClassBinnedRecoPtBinnedRecoNjets2p5_out", inputDir='./classifiers')
ut.setParams()
effFitter = ut.loadOrMake()

entered config files named my_train_config
None
Load object with the name effGenVarClassBinnedRecoPtBinnedRecoNjets2p5_out and the following paramters 
{'classifiers': [],
 'clean': [],
 'dataDir': './data',
 'dataFname': 'output_InsideAcceptance_125.root',
 'defineBins': {},
 'forceMake': False,
 'genBranches': ['genPt',
                 'genRapidity',
                 'genJet2p5Pt0',
                 'genJet2p5Rapidity0',
                 'genJet2p5Pt1',
                 'genJet2p5Rapidity1',
                 'genJet2p5Pt2',
                 'genJet2p5Rapidity2',
                 'genJet2p5Pt3',
                 'genJet2p5Rapidity3',
                 'weight',
                 'genNjets2p5'],
 'inputDir': './classifiers',
 'inputName': 'effGenVarClassBinnedRecoPtBinnedRecoNjets2p5_out',
 'load': True,
 'ncats': 3,
 'outDir': '.',
 'outName': 'effFitter_out',
 'pfx': 'genDiphotonDumper/trees/InsideAcceptance_125_13TeV',
 'recoBranches': ['recoPt', 'recoRapidity', 'recoNjets2p5'],
 'rn

### define bins in genJet2p5Rapidity0 and genJet2p5Pt0

In [3]:
 defineBins = { 'genJet2p5Rapidity0' : dict(boundaries=[-5.,-2.,0.,2.,5.],overflow=False), # do not add overflow automatically
                'genJet2p5Pt0' : dict(boundaries=[0.,50.,100.,200.,1000.],overflow=False)
            }   

In [4]:
ut.runDefineBins(effFitter,defineBins)

### Extract data frame and start getting the probas in a binned way

In [5]:
df = effFitter.df
first_train_evt = int(round(df.index.size*(1.-effFitter.split_frac)))
df_test = df[:first_train_evt]

In [6]:
def weighted_average(df_name, column_name, weight_name=None):
    """
    This function computes the weighted average of the quantity column_name
    stared in the pandas dataframe df_name. In case no weights are given
    or if they sum up to zero, the mean is returned instead.
    :params 
            df_name :
        column_name :
        weight_name :
    :retruns
                    :
    """
    d = df_name[column_name]
    w = df_name[weight_name]
    if (weight_name == None) :
        return float(d.mean())
    else :
        try:
            return (d * w).sum() / float(w.sum())
        except ZeroDivisionError:
            return float(d.mean())

In [7]:
df_test.groupby(['genJet2p5Rapidity0Bin','genJet2p5Pt0Bin']).apply(weighted_average,"recoNjets2p5Cat_prob_0",'absweight')

genJet2p5Rapidity0Bin  genJet2p5Pt0Bin
0                      0                  0.391216
                       1                  0.397270
                       2                  0.363839
                       3                  0.345287
1                      0                  0.410543
                       1                  0.406267
                       2                  0.382631
                       3                  0.335750
2                      0                  0.412034
                       1                  0.407390
                       2                  0.382579
                       3                  0.343419
3                      0                  0.397346
                       1                  0.408442
                       2                  0.374781
                       3                  0.323438
dtype: float64

### get the "true" probability in each bin by computing the frequency of each event

In [9]:
print(df_test['genJet2p5Rapidity0Bin'].min())
print(df_test['genJet2p5Rapidity0Bin'].max())
print(df_test['genJet2p5Pt0Bin'].min())
print(df_test['genJet2p5Pt0Bin'].max())


0
3
nan
3


In [11]:
df_test[['genJet2p5Pt0Bin','genJet2p5Pt0']][0:5]

,genJet2p5Pt0Bin,genJet2p5Pt0
random_index,,
0,1.0,84.751411
1,NaN,0.000000
2,0.0,41.552631
3,0.0,47.969078
4,1.0,68.500809
